In [ ]:
from tinygrad.tensor import Tensor
import numpy as np
from torch.utils.data import DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

from tqdm import tqdm
from glob import glob
from PIL import Image
from random import shuffle

from dataset import CatVsDogsDataset

In [ ]:
BATCH_SIZE = 16

In [ ]:
classes = ["Cat", "Dog"]

In [ ]:
transform = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize()
    ]
)

In [ ]:
train_dataset = CatVsDogsDataset(
    train_images, classes=classes, transform=transform
)
val_dataset = CatVsDogsDataset(val_images, classes=classes, transform=transform)

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=1)

In [ ]:
for X, y, _ in train_dataloader:
    print(y)
    break

In [ ]:
import tinygrad.nn as nn
import tinygrad.nn.optim as optim
from tinygrad.tensor import Tensor

In [ ]:
class CatsVsDogsSimpleNet:
    def __init__(self):
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=0
        )
        self.conv2 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0
        )
        self.conv3 = nn.Conv2d(
            in_channels=64, out_channels=128, kernel_size=5, stride=2, padding=0
        )
        self.fc1 = nn.Linear(128 * 3 * 3, 2)

    def __call__(self, x):
        x = self.conv1(x)
        x = x.relu()
        x = Tensor.max_pool2d(x, [2, 2])
        x = self.conv2(x)
        x = x.relu()
        x = Tensor.max_pool2d(x, [2, 2])
        x = self.conv3(x)
        x = x.relu()
        x = Tensor.max_pool2d(x, [8, 8])
        x = x.flatten(1)
        x = self.fc1(x)

        return x

In [ ]:
model = CatsVsDogsSimpleNet()

In [ ]:
parameters = optim.get_parameters(model)

In [ ]:
sum(map(lambda x: np.prod(np.array(x.realize().shape)), parameters))

In [ ]:
parameters[1].realize().shape

In [ ]:
optimizer = optim.SGD(parameters, lr=1e-3)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
NUM_EPOCHS = 1

In [ ]:
def sparse_categorical_crossentropy(out, Y):
    num_classes = out.shape[-1]
    YY = Y.flatten()
    y = np.zeros((YY.shape[0], num_classes), np.float32)
    y[range(y.shape[0]),YY] = -1.0*num_classes
    y = y.reshape(list(Y.shape)+[num_classes])
    y = Tensor(y)
    return out.mul(y).mean()

In [ ]:
for epoch in range(NUM_EPOCHS):
    Tensor.training = True
    for X, y, _ in train_dataloader:
        X = Tensor(np.moveaxis(X.numpy(), -1, 1), requires_grad=False).cpu()
        y = y.numpy()

        optimizer.zero_grad()
        
        out = model(X)
        loss = sparse_categorical_crossentropy(out, y)
#         loss = out.logsoftmax().mul(y).mean()
        loss.backward()
        
        print(loss.realize())
        
        optimizer.step()
        

In [ ]:
# realize

In [ ]:
Tensor.randn(16, 3, 224, 224, requires_grad=False)